In [2]:
import cv2
import numpy as np
from keras.models import load_model
import pickle
from mtcnn.mtcnn import MTCNN

Using TensorFlow backend.


In [3]:
detector = MTCNN()
predict_model = pickle.load(open('predict_model.sav', 'rb'))

W0709 22:44:14.157736 139916862644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/mtcnn/mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0709 22:44:14.184084 139916862644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/mtcnn/mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0709 22:44:14.233044 139916862644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/mtcnn/network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0709 22:44:14.234106 139916862644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/mtcnn/layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 22:44:14.237669 139916862644544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/mtcnn/layer_factory.py:7

In [8]:
image = cv2.imread("testobama.jpg",1)

print (image.shape)
result = detector.detect_faces(image)
for person in result:
    print (person)
    bounding_box = person['box']
    keypoints = person['keypoints']
    x = bounding_box[1]
    y = bounding_box[0]
    w = bounding_box[2]
    h = bounding_box[3]
    cv2.rectangle(image, (bounding_box[0], bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(0,155,155),2)
cv2.imshow("t",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(600, 1200, 3)
{'box': [432, 238, 113, 134], 'confidence': 0.9999986886978149, 'keypoints': {'left_eye': (465, 288), 'right_eye': (515, 290), 'nose': (491, 315), 'mouth_left': (466, 335), 'mouth_right': (516, 337)}}
{'box': [181, 162, 109, 142], 'confidence': 0.9999561309814453, 'keypoints': {'left_eye': (220, 216), 'right_eye': (273, 221), 'nose': (249, 247), 'mouth_left': (214, 265), 'mouth_right': (266, 270)}}
{'box': [740, 171, 113, 144], 'confidence': 0.9996675252914429, 'keypoints': {'left_eye': (780, 224), 'right_eye': (833, 223), 'nose': (812, 256), 'mouth_left': (775, 273), 'mouth_right': (831, 273)}}
{'box': [994, 217, 100, 128], 'confidence': 0.9996657371520996, 'keypoints': {'left_eye': (1021, 270), 'right_eye': (1068, 268), 'nose': (1043, 292), 'mouth_left': (1023, 315), 'mouth_right': (1067, 315)}}


In [ ]:
# dung mtcnn de du doan
# voi moi ket qua, resize thanh img 160x160x3
# chuyen cac anh thanh dang embedding vector
# du doan
# thresh old < 50: strangeger